# IO编程




IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

比如你打开浏览器，访问新浪首页，浏览器这个程序就需要通过网络IO获取新浪的网页。浏览器首先会发送数据给新浪服务器，告诉它我想要首页的HTML，这个动作是往外发数据，叫Output，随后新浪服务器把网页发过来，这个动作是从外面接收数据，叫Input。所以，通常，程序完成IO操作会有Input和Output两个数据流。当然也有只用一个的情况，比如，从磁盘读取文件到内存，就只有Input操作，反过来，把数据写到磁盘文件里，就只是一个Output操作。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和新浪服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

同步和异步的区别就在于是否等待IO执行的结果。好比你去麦当劳点餐，你说“来个汉堡”，服务员告诉你，对不起，汉堡要现做，需要等5分钟，于是你站在收银台前面等了5分钟，拿到汉堡再去逛商场，这是同步IO。

你说“来个汉堡”，服务员告诉你，汉堡需要等5分钟，你可以先去逛商场，等做好了，我们再通知你，这样你可以立刻去干别的事情（逛商场），这是异步IO。

很明显，使用异步IO来编写程序性能会远远高于同步IO，但是异步IO的缺点是编程模型复杂。想想看，你得知道什么时候通知你“汉堡做好了”，而通知你的方法也各不相同。如果是服务员跑过来找到你，这是回调模式，如果服务员发短信通知你，你就得不停地检查手机，这是轮询模式。总之，异步IO的复杂度远远高于同步IO。

操作IO的能力都是由操作系统提供的，每一种编程语言都会把操作系统提供的低级C接口封装起来方便使用，Python也不例外。我们后面会详细讨论Python的IO编程接口。

注意，本章的IO编程都是同步模式。

## 文件读写

读写文件是最常见的IO操作。Python内置了读写文件的函数，用法和C是兼容的。

读写文件前，我们先必须了解一下，在磁盘上读写文件的功能都是由操作系统提供的，现代操作系统不允许普通的程序直接操作磁盘，所以，读写文件就是请求操作系统打开一个文件对象（通常称为文件描述符），然后，通过操作系统提供的接口从这个文件对象中读取数据（读文件），或者把数据写入这个文件对象（写文件）。

## 读文件

要以读文件的模式打开一个文件对象，使用Python内置的open()函数，传入文件名和标示符：

In [1]:
f = open('test.txt', 'r')

标示符'r'表示读，这样，我们就成功地打开了一个文件。

如果文件不存在，open()函数就会抛出一个IOError的错误，并且给出错误码和详细的信息告诉你文件不存在：

In [2]:
f=open('/Users/michael/notfound.txt', 'r')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/michael/notfound.txt'

如果文件打开成功，接下来，调用read()方法可以一次读取文件的全部内容，Python把内容读到内存，用一个str对象表示

In [3]:
f = open('test.txt', 'r')
f.read()

'blablalalal\n12345656\n'

最后一步是调用close()方法关闭文件。文件使用完毕后必须关闭，因为文件对象会占用操作系统的资源，并且操作系统同一时间能打开的文件数量也是有限的：

In [4]:
f.close()

由于文件读写时都有可能产生IOError，一旦出错，后面的f.close()就不会调用。所以，为了保证无论是否出错都能正确地关闭文件，我们可以使用try ... finally来实现：

In [5]:
try:
    f = open('test.txt', 'r')
    print(f.read())
finally:
    if f:
        f.close()

blablalalal
12345656



但是每次都这么写实在太繁琐，所以，Python引入了with语句来自动帮我们调用close()方法：

In [6]:
with open('test.txt', 'r') as f:
    print(f.read())

blablalalal
12345656



这和前面的try ... finally是一样的，但是代码更佳简洁，并且不必调用f.close()方法。

调用read()会一次性读取文件的全部内容，如果文件有10G，内存就爆了，所以，要保险起见，可以反复调用read(size)方法，每次最多读取size个字节的内容。另外，调用readline()可以每次读取一行内容，调用readlines()一次读取所有内容并按行返回list。因此，要根据需要决定怎么调用。

如果文件很小，read()一次性读取最方便；如果不能确定文件大小，反复调用read(size)比较保险；如果是配置文件，调用readlines()最方便：

In [8]:
with open('test.txt', 'r') as f:
    
    for line in f.readlines():
        print(line.strip()) # 把末尾的'\n'删掉

blablalalal
12345656


## file-like Object

像open()函数返回的这种有个read()方法的对象，在Python中统称为file-like Object。除了file外，还可以是内存的字节流，网络流，自定义流等等。file-like Object不要求从特定类继承，只要写个read()方法就行。

StringIO就是在内存中创建的file-like Object，常用作临时缓冲。



## 二进制文件

前面讲的默认都是读取文本文件，并且是UTF-8编码的文本文件。要读取二进制文件，比如图片、视频等等，用'rb'模式打开文件即可：

In [9]:
f = open('python.png', 'rb')

In [10]:
f.read()

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xe1\x00\x00\x00\xe1\x08\x03\x00\x00\x00\tm"H\x00\x00\x00\xc0PLTE\xff\xff\xff\x0c\x9d\xbf\xf8\xb7!\xf8\xb3\x00\x00\x99\xbe\xfd\xe8\xbe\x1f\x9e\xae\x04\x9c\xc0\xf8\xb5\x00\xf4\xf8\xf3\xfe\xf7\xeb\xff\xfa\xf1\xf8\xba$\x9b\xc9\xc7&\xa0\xb5\xfe\xf6\xe3\x82\xbe\xc0\xfb\xd3\x85\xa8\xd0\xce\xea\xf4\xf4\xf9\xbe4\x00\x98\xb7\xb9\xd8\xd2\xff\xfe\xfa\xfd\xe9\xc3\xf8\xb5\x13\xf9\xbe<\x00\x96\xb8\xfc\xe2\xb0\xfd\xe5\xb6\xfa\xc7\\\xfa\xcfy\xe1\xed\xe7\xfe\xf2\xdb\xfc\xdc\x9f\xfb\xd7\x90\x00\x9b\xb4\xfa\xc5V\xa5\xd1\xdf\xfc\xdd\xa8\xfb\xd6\x93\xfa\xc3ME\xa9\xbem\xb7\xc1\x8b\xc5\xd0\x87\xc3\xd5\xfd\xee\xcc\xcc\xe3\xe6i\xb6\xc6]\xb1\xb9\xfa\xcds\xcc\xe2\xdd\xfa\xc9gD\xaa\xc2\xa3\xce\xce\xd6\xe8\xdf\xda\xea\xe9q\xb9\xc5\x8a\xc2\xc4D\xa7\xb3\xdc\xe9\xe0\xaa\xd3\xd8\xc4\xdd\xd5i\xb6\xc4\xef\xad\x8e\x95\x00\x00\x05\xe4IDATx\x9c\xed\x9ckW\xe2:\x14\x86\xa1\xa9\x96j\x81A.%\xe0\x05\x01e\x803:\x8a\xc3\x80\x1e\xf5\xff\xff\xabC/\x80h\x13RN\xe8&\xae\xf7\xf940,V\x1ew

In [11]:
f.close()

## 字符编码

要读取非UTF-8编码的文本文件，需要给open()函数传入encoding参数，例如，读取GBK编码的文件：
```
>>> f = open('/Users/michael/gbk.txt', 'r', encoding='gbk')
>>> f.read()
'测试'
```
遇到有些编码不规范的文件，你可能会遇到UnicodeDecodeError，因为在文本文件中可能夹杂了一些非法编码的字符。遇到这种情况，open()函数还接收一个errors参数，表示如果遇到编码错误后如何处理。最简单的方式是直接忽略：
```
>>> f = open('/Users/michael/gbk.txt', 'r', encoding='gbk', errors='ignore')
```

## 写文件

写文件和读文件是一样的，唯一区别是调用open()函数时，传入标识符'w'或者'wb'表示写文本文件或写二进制文件：

In [12]:
f = open('test2.txt', 'w')

In [13]:
f.write('Hello, world!')

13

In [14]:
f.close()

你可以反复调用write()来写入文件，但是务必要调用f.close()来关闭文件。当我们写文件时，操作系统往往不会立刻把数据写入磁盘，而是放到内存缓存起来，空闲的时候再慢慢写入。只有调用close()方法时，操作系统才保证把没有写入的数据全部写入磁盘。忘记调用close()的后果是数据可能只写了一部分到磁盘，剩下的丢失了。所以，还是用with语句来得保险：

In [15]:
with open('test2.txt', 'w') as f:
    f.write('Hello, world!')

要写入特定编码的文本文件，请给open()函数传入encoding参数，将字符串自动转换成指定编码。

细心的童鞋会发现，以'w'模式写入文件时，如果文件已存在，会直接覆盖（相当于删掉后新写入一个文件）。如果我们希望追加到文件末尾怎么办？可以传入'a'以追加（append）模式写入。

所有模式的定义及含义可以参考Python的官方文档。

## 练习

请将本地一个文本文件读为一个str并打印出来：

In [16]:
fpath = r'test.txt'

with open(fpath, 'r') as f:
    s = f.read()
    print(s)

blablalalal
12345656



## 小结

在Python中，文件读写是通过open()函数打开的文件对象完成的。使用with语句操作文件IO是个好习惯。